### Importamos las librerias con que vamos a trabajar a lo largo de este proyecto

In [1]:
import pandas as pd 
import numpy as np
import ast
import json
import emoji
import re
import nltk
# Esto sirve para configurar NLTK. La primera vez puede tardar un poco
nltk.download('punkt')
nltk.download('stopwords')

### Cargamos los datos
Para realizar la carga de los datos debemos de realizar el siguiente proceso debido a que los archivos estan en formato json no lo podemos importar directamente con la libreria de pandas

In [2]:
rows = []

with open ('australian_user_reviews.json', encoding='utf-8') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df_reviews = pd.DataFrame(rows)

In [ ]:
rows = []

with open ('australian_users_items.json', encoding='utf-8') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df_items = pd.DataFrame(rows)

In [497]:
filas = []

with open("output_steam_games.json", encoding='utf-8') as file:
    for line in file.readlines():
        filas.append(json.loads((line))) # Como los tipo de valores no eran strings que es el tipo esperado por ast.litreal_eval, usamos json.loads

df_games = pd.DataFrame(filas)

Asi se veria nuestro DataFrame df_reviews. Observar que la columna reviews tiene una lista de diccionario por lo cual debemos de realizar una series de pasos para poder desanidar esta columna 

In [6]:
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [7]:
data_desanidada = []

for index, row in df_items.iterrows():
    user_id = row['user_id']
    items_count = row['items_count']
    steam_id = row['steam_id']
    user_url = row['user_url']
    items = row['items']
    
    for i in items:   
        new_row = {
        'user_id': user_id,
        'items_count': items_count,
        'steam_id' : steam_id,
        'user_url' : user_url,
        'item_id': i.get('item_id', ''),
        'item_name': i.get('item_name', ''),
        'playtime_forever': i.get('playtime_forever', ''),
        'playtime_2weeks': i.get('playtime_2weeks', '')
        }
        
        data_desanidada.append(new_row)

df_items_desanidada = pd.DataFrame(data_desanidada)

El paso anterior tambien lo repetimos para el DataFrame de df_reviews en la columna reviews tambien es una lista de diccionario

In [238]:
data_desanidada = []

for index, row in df_reviews.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    
    reviews= row['reviews']
    
    
    for i in reviews:   
        new_row = {
        'user_id': user_id,
        'user_url':user_url,
        
        'funny': i.get('funny', ''),
        'posted': i.get('posted', ''),
        'last_edited': i.get('last_edited', ''),
        'item_id': i.get('item_id', ''),
        'helpful': i.get('helpful', ''),
        'recommend': i.get('recommend', ''),
        'review': i.get('review', '')
        }
        
        data_desanidada.append(new_row)

df_reviews_desanidada=pd.DataFrame(data_desanidada)

In [239]:
len(df_reviews_desanidada[df_reviews_desanidada['review']!=""])

59275

## Analisis de sentimiento
Este analisis lo vamos a realizar para nuestro df_reviews_desanidada el cual tiene las reviews hecha por los usuarios de los video juegos. Para lo cual es necesario que nuestra columna reviews no tenga valores nulos o vacios para lo cual aplicamos una mascara que filtre todos los valores execto los vacios.

In [240]:
## aplicamos una mascara que nos dejen aquellas filas que no son vacias y la guargamos en la variable data_reviews_desanidada 
df_reviews_desanidada=df_reviews_desanidada[df_reviews_desanidada['review']!=""]

Como nuestra columnas de review tiene comentario con emojis es necesarios filtrar todo esto emojis para realizar un tratamiento de datos con el proposito de obtener mayor informacion para realizar nuestro analisis de sentimiento. para lo cual vamos apoyar en una libreria llamada **emoji** 

Lo primero que vamos hacer es crear una funcion (emojis_count(x)) a la cual le vamos a ingresar como parametro (x) un texto y esta a su vez devolvera una lista con los emojis que se encuanta en dicho texto

In [243]:
def emojis_count(x):
    emojis = []
    emojis_encontrados = emoji.emoji_list(x)
    if emojis_encontrados == '':
        return ''
    for e in emojis_encontrados:
        emojis.append(e['emoji'])
    return emojis

Utilizamos la funcion lambda la cual nos va ayudar a ingresar uno a uno de los texto de la columna reviwe en la funcion que creamos en el paso anterios y nos devolvera una lista en caso de no encontrar emojis devuelve una lista vacia

In [244]:
emojis = df_reviews_desanidada['review'].apply(lambda x: emojis_count(x))
emojis

0                     []
1                     []
2                     []
3              [♥, ♥, ♥]
4        [♥, ♥, ♥, ♥, ♥]
              ...       
59300                 []
59301                 []
59302                 []
59303                 []
59304                 []
Name: review, Length: 59275, dtype: object

La siguiente funcion (obtener_emojis_unicos(emojis)) devuelva una lista de emojis unicos es decir eliminando a los emojis repetidos

In [245]:
def obtener_emojis_unicos(emojis):
    return list(set(emojis))

In [ ]:
#####  salto de linea
#####

En el proximo paso hacemos dos procesos:
    1. filtramos las lista vacias dada en el paso anterios
    2. utilizamos la funcion creados en el paso anterior (obtener_emojis_unicos(emojis)) para eliminar aquellos emojis repetidos

In [246]:
emojis = emojis[emojis.apply(lambda x: len(x) > 0)]
emojis = emojis.apply(obtener_emojis_unicos)

Eliminamos los duplicados

In [248]:
emojis.drop_duplicates(inplace=True)
emojis.count()

28

Aqui tenemos todos los emojis que se encontraba en la columna de review

In [250]:
emojis

3                       [♥]
330                     [👌]
452                  [❤, ♥]
509                     [©]
3546                    [❤]
3854                    [®]
4384                    [™]
9144                    [😄]
9549           [❤, ☹, ☑, ☺]
12525               [👈🏻, 😋]
12949                   [😀]
13401                   [✔]
18124                   [😃]
18562       [👌, 💯, 👀, ♥, ✔]
18574                [®, ♥]
20439    [⌛, ❤, ◀, ☑, ☹, ☺]
20568                [™, ♥]
22475                   [💋]
22518                   [👍]
22858                [☑, ♥]
22952                   [♀]
24567                   [🐐]
26071                   [🐥]
33945                   [💀]
42284          [👌, ✔, 💯, 👀]
44187                   [😊]
44736                   [♠]
52523             [☹, ♥, ☺]
Name: review, dtype: object

creamos una lista un diccionario con todos los emojis encontrados y a cada emojis se le asignos una palabra con la cual vamos a remplazar dicho emoji. Adicionamente le pedimos a chatgpt que nos diera ua lista de emojis de signos de puntuacion (:D, :p, :(), <3 ) a los cuales tambien le asignamos una palabra

In [251]:
## creamos una lista con los emojis juanto con las palabra que vamos a utilizar de remplazos

emoji_reemplazos = {
    '✔': 'Approved ',
    '💯': 'Perfect ',
    '♀': 'Woman ',
    '❤': 'Love ',
    '💋': 'Kiss ',
    '💀': 'Skull ',
    '™': 'Trademark ',
    '®': 'Registered Trademark ',
    '©': 'Copyright ',
    '☺': 'Smile ',
    '♥': 'Heart ',
    '👌': 'Okay ',
    '👀': 'Eye ',
    '⌛': 'Hourglass ',
    '☹': 'Sad ',
    '☑': 'Checked ',
    '😊': 'Happiness ',
    '😄': 'Happiness ',
    '◀': 'Left ',
    ':D': 'laugh ',
    ':3': 'smile',
    'xD': 'laughloud',
    'ô•': 'oh',
    ':c': 'adictive',
    ':)': 'happy',
    ':p': 'boring',
    '^^': 'good',
    'D=': 'pain',
    ':(': 'sad',
    'xd': 'happiness',
    '=P': 'crazy',
    '<3': 'heart',
    ';>': 'blink',
    '(Y)': 'thumbs up',
    '---': 'enjoyable',
    ',3': 'amazing',
    'xt': 'unwanted',
    '-' : 'recommended',
    ':}': 'good',
    'c:': 'happy',
    ':v': 'want more',
    'x3': 'cute face',
    'XD': 'adictive',
    '(y)': 'thumbs up',
    ':P': 'bad',
    ';0': 'laugh',
    '=)': 'happy',
    'gg': 'great game',
    'GG': 'great game',
    ';D': 'great game',
    ':c': 'sad'
}

Creamos una funcion la cual va a remplazar cada uno de los emojis por las palabras asinadas

In [252]:
def replace_emojis(text):
    for emoji, replacement in emoji_reemplazos.items():
        text = text.replace(emoji, replacement)
    return text

Aplicamos la funcion replace_emojis(text) a nuestra columna de review

In [253]:
df_reviews_desanidada['review']=df_reviews_desanidada['review'].apply(replace_emojis)

### Importamos las siguentes librerias para hacer nuestro analisis de sentimeto

In [ ]:
# Importamos esta libreria que nos permite reemplzar caracteres
import re

# Importamos la función que nos permite Stemmizar de nltk y definimos el stemmer
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# Traemos nuevamente las stopwords
stopwords = nltk.corpus.stopwords.words('english')


El siguente bloque de codigo lo utilizamos para normalizar nuestra columna de reviews con el fin de poder utilizar nuestra libreria de nltk y poder realizar el analisis de sentimieto

In [262]:
review_list = []

# Itera a través de las filas del DataFrame y su columna 'review'
for index, row in df_reviews_desanidada.iterrows():
    review = row['review']
    
    # Reemplazamos los caracteres que no sean letras por espacios
    review = re.sub("[^a-zA-Z]", " ", str(review))
    
    # Convertimos todo a minúsculas
    reviewr = review.lower()
    
    # Tokenizamos para separar las palabras
    review = nltk.word_tokenize(review)
    
    # Eliminamos las palabras de menos de 1 caracter
    review = [palabra for palabra in review if len(palabra) > 1]
    
    # Sacamos las Stopwords
    review = [palabra for palabra in review if not palabra in stopwords]
    
    # Aplicamos la función para buscar la raíz de las palabras
    review = [stemmer.stem(palabra) for palabra in review]
    
    # Por último, volvemos a unir el titular
    review = " ".join(review)
    
    # Agregamos el titular procesado a la lista
    review_list.append(review)

# Crear una nueva columna en el DataFrame con los titulares procesados
df_reviews_desanidada['review_normalizado'] = review_list

# Imprimir el DataFrame con los titulares procesados
print(df_reviews_desanidada)

                 user_id                                           user_url  \
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
2      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
3                js41637               http://steamcommunity.com/id/js41637   
4                js41637               http://steamcommunity.com/id/js41637   
...                  ...                                                ...   
59300  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
59301  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
59302        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
59303        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
59304        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   

                                  funny            

convertimos la columna recommden a a dummy

In [267]:
dummy_recommend=pd.get_dummies(df_reviews_desanidada["recommend"], prefix="recommend").astype(int)

In [269]:
df_reviews_desanidada=pd.concat([df_reviews_desanidada,dummy_recommend], axis=1)

## Procesamiento de los sentimiento de la columna review

Importamos la libreria nlyk para analizar los sentimieto de nuestra columna reviews

In [278]:
## libreria para analizar los sentimiento
from nltk.sentiment.vader import SentimentIntensityAnalyzer

Opcional: En caso de que la libreria nltk no pueda cargar el archivo vader usar el siguiente codigo

In [282]:

import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...


True

#### A continuacion daremos un breve ejemplo de como la libreria nltk analisa los snetimientos

texto a analizar

In [288]:

x="it is a charming and beautiful produc"

Instanciamos el analizadosr de texto y analizamos el texto anterior

In [289]:
sid = SentimentIntensityAnalyzer()  # Instanciamos el analizados de texto
resultado= sid.polarity_scores(x) # analizamos el texto

Observece que no deuelve la probabilidad de que el texto tenga sentimiento negativo, sentimiento neutro y sentimiento positivo. En esta caso sentimiento positivo tiene una probo de 0.658. tambien lo podemos analizar con la imformacion que nos proporciona el compound el cual nos indica que si es mayor a 0 el texto tiene sentimiento positivio, si el compound es menor a 0 el sentimiento del texto es negativo y si el compound es igual a 0 el sentimieto del texto es neutro

In [290]:
resultado

{'neg': 0.0, 'neu': 0.342, 'pos': 0.658, 'compound': 0.8271}

Aplicamos la funcion lambda para que a cada uno de lo elemento de la columna review_normalizado le aplique el analisis de sentimiento y nos devuelva como resultado solo el compound y la va almancenar en una nueva columna de nombre sentiment_analysis

In [340]:
df_reviews_desanidada['sentiment_analysis']=df_reviews_desanidada["review_normalizado"].apply(lambda i: sid.polarity_scores(i)['compound'])

In [ ]:
### 'compound'>0 sentimieto positivo
### 'compound'< Sentimiento negativo
### 'compound' == 0 sentimiento neutro

Remplazamos los valores de las columna 'sentiment_analysis' por los parametros dados en el readme del proyecto

- NOTA: tener en cuenta el orden en que se realiza los remplazos debido a que podemos trocar los valores

In [345]:
df_reviews_desanidada.loc[df_reviews_desanidada['sentiment_analysis']>0,'sentiment_analysis']=2
df_reviews_desanidada.loc[df_reviews_desanidada['sentiment_analysis']==0,'sentiment_analysis']=1
df_reviews_desanidada.loc[df_reviews_desanidada['sentiment_analysis']<0,'sentiment_analysis']=0

## Realizamos el ETL

Guardamos la data de df_reviews_desanidada en la variable data_review por si cometemos algun error no tengamos que volver a cargar la data y repetir todos los procesos anterioes

In [450]:
data_review=df_reviews_desanidada

Eliminamos las siguientes columnas debido a que tiene demaciados valores vacios

In [374]:
### eliminamos esta dos columnas 
###

print("el numero de filas vacias en la columna funny es:",len(data_review[data_review['funny']=='']))
print("el numero de filas vacias en la columna last_edited es:",len(data_review[data_review['last_edited']=='']))


el numero de filas vacias en la columna funny es: 51125
el numero de filas vacias en la columna last_edited es: 53135


In [375]:
data_review.drop(columns=['funny','last_edited'], inplace=True)

### Tambien vamos a eliminar las siguientes columnas

- helpful:  no aporta mucha informacion relevante
- recommend: esta columna la convertimos a dummy (recommend_False, recommend_True)
- review y review_normalizado por que estan remplazadas por la columna sentiment_analysis

In [381]:
data_review.drop(columns=['helpful','recommend','review','review_normalizado'], inplace=True)

#### Guardamos la variable df_items_desanidada en la variable data_items por si cometemos algun error no tengamos que volver a cargar la data

In [410]:
data_items=df_items_desanidada

#### Guardamos la variable df_games en la variable data_games por si cometemos un error no tengamos que volver a cargar la data

In [498]:
data_games=df_games

Eliminamos las filas que tienes mas de 5 valores faltantes

In [499]:
data_games.dropna(inplace=True, thresh=5)

In [466]:
data_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL


In [ ]:
###### REALIZAMOS La SIGUIENTE FUNCION
######  
######   def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, 
######                                  el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [501]:
## observamos la lista con valores unicos y mirams que hay precio que no tiene valor en numeros
data_games['price'].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, nan, 'Free', 10.99, 1.59, 14.99, 1.99, 59.99, 8.99,
       6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49, 15.99, 1.25,
       24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.39, 'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0,
       9.0, 234.99, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99,
       179.0, 26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 199.99,
       22.99, 320.0, 38.85, 71.7, 59.95, 995.0, 27.49, 3.39, 6.0, 19.95,
       499.99, 16.06, 4.68, 131

In [ ]:
#no fecha
#no genero 

In [510]:
## creamos una lista de precios a remplazar
remplazos_precio={'Free To Play': 0,
                  'Free to Play':0,
                   'nan':0,
                  'Free':0,
                  'Free Demo':0,
                  'Play for Free!':0,
                  'Install Now':0,
                  'Play WARMACHINE: Tactics Demo':0,
                  'Free Mod':0,
                  'Install Theme':0,
                  'Third-party':0,
                  'Play Now':0,
                  'Free HITMAN™ Holiday Pack':0,
                  'Play the Demo':0,
                  'Starting at $499.00':499,
                  'Starting at $449.00':499,
                  'Free to Try':0,
                  'Free Movie':0,
                  'Free to Use':0
                                                 
}

In [508]:
## realizamos el premplazo de precios
data_games['price']=data_games['price'].replace(remplazos_precio)

In [ ]:
lista_juego=data_items[data_items['user_id']=='76561197970982479']['item_id'].tolist()
lista_juego

In [598]:
data_items[data_items['user_id']=='76561197970982479']['item_id'].count()

277

In [567]:
### juanaaaaaa
lista_precio=[]
for i in lista_juego:

    lista_precio.append(data_games[data_games['id']==i]["price"].tolist())
    
print(lista_precio)    

[[9.99], [4.99], [4.99], [4.99], [4.99], [4.99], [9.99], [4.99], [9.99], [19.99], [9.99], [19.99], [19.99], [], [9.99], [9.99], [6.99], [19.99], [19.99], [9.99], [4.99], [nan], [9.99], [7.99], [9.99], [7.99], [], [], [4.99], [19.99], [19.99], [9.99], [9.99], [9.99], [2.99], [2.99], [19.99], [], [9.99], [19.99], [19.99], [19.99], [19.99], [19.99], [9.99], [14.99], [19.99], [0.0], [4.99], [4.99], [4.99], [], [14.99], [9.99], [], [19.99], [9.99], [9.99], [9.99], [19.99], [9.99], [], [], [19.99], [], [19.99], [14.99], [9.99], [19.99], [], [19.99], [9.99], [6.99], [], [19.99], [19.99], [19.99], [6.99], [19.99], [9.99], [19.99], [19.99], [], [14.99], [9.99], [], [], [], [], [], [54.99], [], [], [], [], [19.99], [], [19.99], [], [29.99], [4.99], [4.99], [19.99], [], [29.99], [14.99], [9.99], [9.99], [9.99], [14.99], [9.99], [39.99], [], [4.99], [14.99], [9.99], [9.99], [], [9.99], [19.99], [0.0], [14.99], [2.99], [4.99], [9.99], [], [4.99], [], [19.99], [9.99], [9.99], [], [9.99], [9.99], [9.

In [593]:
lista_juego=data_items[data_items['user_id']=='76561197970982479']['item_id'].tolist()

import math  # Importa el módulo math para verificar si un valor es nulo
lista_precio=[]

for i in lista_juego:

    lista_precio.append(data_games[data_games['id']==i]["price"].tolist())

suma = 0  # contador de suma

for i in lista_juego:
    prices = data_games[data_games['id'] == i]["price"].tolist()
    
    # Verifica si la lista de precios no está vacía
    if prices:
        # Filtra los valores no numéricos y nan y luego suma los valores
        validar_prices = [p for p in prices if isinstance(p, (int, float)) and not math.isnan(p)]
        suma += sum(validar_prices)  # Suma los valores válidos de la lista de precios

print('La cantidad de dinero gastado por es usuario es de:',suma)


cantidad_items=data_items[data_items['user_id']=='76561197970982479']['item_id'].count()
cantidad_review=data_review[data_review['user_id']=='76561197970982479']['sentiment_analysis'].count()
print('El porcentaje de recomendacion es:', (cantidad_review/cantidad_items)*100)


La cantidad de dinero gastado por es usuario es de: 3424.3099999999854
El porcentaje de recomendacion es: 1.083032490974729


In [ ]:
#data_games
#data_items
#data_review

In [601]:
data_review.head(2)

,user_id,user_url,posted,item_id,recommend_False,recommend_True,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,0,1,2.0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted July 15, 2011.",22200,0,1,2.0


In [605]:
len(data_review['posted'].unique())

1906

In [600]:
data_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,Secret Level SRL


## Exportamos nuestros tres dataFrames para realizar las funciones

In [606]:
## exportamos los dataFrames
data_games.to_csv('games.csv', index=False)
data_items.to_csv('items.csv', index=False)
data_review.to_csv('reviews.csv', index=False)